## User Management
Interaction with EOEPCA endpoint using python Demo Client.

In [ ]:
import utils.DemoClient as client
import jwt
import json
import requests

## Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [ ]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "demo.eoepca.org"
platform_domain = "test." + base_domain
base_url = "https://" + platform_domain

authorize_url = 'https://'+platform_domain+'/oxauth/restv1/authorize'
token_url = 'https://'+platform_domain+'/oxauth/restv1/token'
demo = client.DemoClient(base_url)
demo.register_um_client()
demo.save_state()

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

### Authenticate

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="demoA"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)
print("user_id_token:", user_id_token)

### Inspect the ID Token

In [ ]:
# Inspect ID Token
jwt_header = jwt.get_unverified_header(user_id_token)
jwt_payload = jwt.decode(user_id_token, options={"verify_signature": False})
print("JWT Header:", json.dumps(jwt_header, indent = 2), "\n---\nJWT Payload:", json.dumps(jwt_payload, indent = 2))

The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.

## External ID Provider Login

The current External IdPs are GitHub and COIH, this test will apply the authorization code flow of OAuth2, where a first request to the authorization endpoint of the Login Service will return a code once the external user is authenticated. Afterwards the redirection will prompt a GitHub logo Image but in the url we can extract the code,that will be used to retrieve an access token. Copy the URL and proceed to next step

In [ ]:
from rauth import OAuth2Service
client_id, client_secret = demo.get_client_credentials()

scope = ['openid', 'email', 'user_name', 'is_operator']
redirect_uri =base_url+ "/oxauth/auth/passport/img/github.png"
code = None

service = OAuth2Service(name="foo", client_id=client_id, client_secret=client_secret, access_token_url=token_url, authorize_url=authorize_url, base_url=base_url)
params = {'scope': scope,
          'response_type': 'code',
          'redirect_uri': redirect_uri,
          'acr_values': 'passport_social'}

url = service.get_authorize_url(**params)
print("Link to Social Passport Login: ")
print(url)

Here you will have to copy/paste the redirection once logged in to retrieve code and continue with the flow. An access token should result as output

In [ ]:
code_url = input('Please paste redirect URL: ').strip()
code = code_url.split('code=')[1].split('&')[0]
data = {
  'client_id': client_id,
  'client_secret': client_secret,
  'code': code,
  'grant_type': 'authorization_code',
  'redirect_uri': redirect_uri
}
response = requests.post(token_url, data=data)
print(response.text)

## Access Protected Endpoint
We perform an example access to an ADES resource.
The ADES is protected by a PEP (Policy Enforcement Point). When accessing the protected endpoint the client follows a UMA (User Managed Access) flow to obtain authorized access on behalf of the user. The UMA flow utilises the ID Token to obtain a 'ticket' which can then be exchanged by the client for an access token (RPT - Relying Party Token) that acts as a short-lived (e.g. 5 mins) credential for the specific access by a specific client, to a specific resource, on behalf of a specific user.

In [ ]:
#--------------------------
#GET TICKET
#--------------------------
ades_res_proc_url = "http://ades-pep." + base_domain + "/authorize" ; print("ADES API Processes endpoint:", ades_res_proc_url)
headers = { 'content-type': "application/json", "cache-control": "no-cache", "Authorization": "Bearer "+user_id_token}

tkt = requests.get(ades_res_proc_url, headers=headers, verify=False)
ticket = tkt.headers["WWW-Authenticate"].split("ticket=")[1]
print("401 Unauthorized, ticket returned: "+ticket)

Request RPT token from UMA endpoints giving a ticket and the user id_token

In [ ]:
#--------------------------
#GET RPT
#--------------------------
token_ep= base_url + "/oxauth/restv1/token"
client_id, client_secret = demo.get_client_credentials()
data = "claim_token_format=http://openid.net/specs/openid-connect-core-1_0.html#IDToken&claim_token="+user_id_token+"&ticket="+ticket+"&grant_type=urn%3Aietf%3Aparams%3Aoauth%3Agrant-type%3Auma-ticket&client_id="+client_id+"&client_secret="+client_secret+"&scope=openid"
headers= { 'Content-Type': 'application/x-www-form-urlencoded', 'cache-control': 'no-cache' }
rpts_response = requests.post(token_ep, data=data, headers=headers, verify = False)
rpt = rpts_response.json()["access_token"]
print("The UMA returns: " + str(rpts_response))
print("The RPT Token: "+ str(rpt))

The access_token is returned and the access to the resource is retried with aproved credentials

In [ ]:
#--------------------------
#ACCESS TO RESOURCE (NO TICKET RETURNED)
#--------------------------
headers = { 'content-type': "application/json", "cache-control": "no-cache", "Authorization": "Bearer "+rpt}
tkt = requests.get(ades_res_proc_url, headers=headers, verify=False)
print("Retrying access to resource with proper token: "+ str(tkt))